# Stacking Ensemble Model

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
cancer_data = load_breast_cancer()

X_data = cancer_data.data
y_label = cancer_data.target

X_train, X_test, y_train, y_test = train_test_split(X_data, y_label, test_size=0.2, random_state=0)

In [7]:
# 개별 ML 모델 생성
knn_clf = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=100)

#Stacking Model
lr_final = LogisticRegression(C=10)

In [8]:
knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train, y_train)
dt_clf.fit(X_train, y_train)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=100, random_state=None)

In [9]:
knn_pred = knn_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)

In [11]:
print('KNN:{}'.format(accuracy_score(y_test, knn_pred)))
print('randomforest:{}'.format(accuracy_score(y_test, rf_pred)))
print('decisiontree:{}'.format(accuracy_score(y_test, dt_pred)))
print('adaboost:{}'.format(accuracy_score(y_test, ada_pred)))


KNN:0.9210526315789473
randomforest:0.9649122807017544
decisiontree:0.9035087719298246
adaboost:0.956140350877193


In [12]:
pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
print(pred.shape)

(4, 114)


In [14]:
pred = np.transpose(pred)
print(pred.shape)

(114, 4)


In [20]:
lr_final.fit(pred, y_test)
final = lr_final.predict(pred)

In [16]:
len(X_test)

114

In [21]:
print('final meta model{}'.format(accuracy_score(y_test, final)))

final meta model0.9649122807017544


## CV Stacking

In [22]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds):
    kf = KFold(n_splits=n_folds, shuffle=False, random_state=0)
    train_fold_pred = np.zeros((X_train_n.shape[0], 1))
    test_pred = np.zeros((X_test_n.shape[0], n_folds))
    print(model.__class__.__name__, 'model 시작')
    
    for folder_counter, (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        print('\t fold set:', folder_counter, '시작')
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index]
        
        model.fit(X_tr, y_tr)
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1, 1)
        test_pred[:, folder_counter] = model.predict(X_test_n)
        
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1, 1)
    
    return train_fold_pred, test_pred_mean

In [25]:
knn_train, knn_test = get_stacking_base_datasets(knn_clf, X_train, y_train, X_test, 7)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, X_train, y_train, X_test, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, X_train, y_train, X_test, 7)
ada_train, ada_test = get_stacking_base_datasets(ada_clf, X_train, y_train, X_test, 7)

KNeighborsClassifier model 시작
	 fold set: 0 시작
	 fold set: 1 시작
	 fold set: 2 시작
	 fold set: 3 시작
	 fold set: 4 시작
	 fold set: 5 시작
	 fold set: 6 시작
RandomForestClassifier model 시작
	 fold set: 0 시작
	 fold set: 1 시작
	 fold set: 2 시작
	 fold set: 3 시작
	 fold set: 4 시작
	 fold set: 5 시작
	 fold set: 6 시작
DecisionTreeClassifier model 시작
	 fold set: 0 시작
	 fold set: 1 시작
	 fold set: 2 시작
	 fold set: 3 시작
	 fold set: 4 시작
	 fold set: 5 시작
	 fold set: 6 시작
AdaBoostClassifier model 시작
	 fold set: 0 시작
	 fold set: 1 시작
	 fold set: 2 시작
	 fold set: 3 시작
	 fold set: 4 시작
	 fold set: 5 시작
	 fold set: 6 시작


In [26]:
Stack_final_X_train = np.concatenate((knn_train, rf_train, dt_train, ada_train), axis=1)
Stack_final_X_test = np.concatenate((knn_test, rf_test, dt_test, ada_test), axis=1)

In [27]:
lr_final.fit(Stack_final_X_train, y_train)
stack_final = lr_final.predict(Stack_final_X_test)

print(accuracy_score(y_test, stack_final))

0.9736842105263158
